In [1]:
from copy import deepcopy
import itertools
import jsonlines
from pathlib import Path
import random

from sklearn.metrics import classification_report
from sklearn.metrics import cohen_kappa_score

from tasks.contrastive_conditioning import MucowContrastiveConditioningTask
from translation_models.fairseq_models import load_sota_evaluator
from translation_models.testing_models import DictTranslationModel

In [2]:
# Define data paths
mucow_path = Path(".").parent.parent.parent / "data" / "mucow"
data_path = Path(".") / "data"
pattern_matching_log_path = data_path / "mucow_pattern_matching.results.en-de.ensemble.log"
mucow_ende_annotator1_path = data_path / "en-de.annotator1.jsonl"
mucow_ende_annotator2_path = data_path / "en-de.annotator2.jsonl"

In [3]:
# Load annotations
with jsonlines.open(mucow_ende_annotator1_path) as f:
  annotations1 = {line["Sample ID"]: line for line in f}
with jsonlines.open(mucow_ende_annotator2_path) as f:
  annotations2 = {line["Sample ID"]: line for line in f}

# Flatten labels
for key in annotations1:
    annotations1[key]["label"] = annotations1[key]["label"][0]
for key in annotations2:
    annotations2[key]["label"] = annotations2[key]["label"][0]

In [4]:
# Remove samples that were only partially annotated
for key in list(annotations1.keys()):
    if key not in annotations2:
        del annotations1[key]
for key in list(annotations2.keys()):
    if key not in annotations1:
        del annotations2[key]
print(f"Number of annotations: {len(annotations1)} + {len(annotations2)}")

Number of annotations: 185 + 185


In [5]:
# Inter-annotator agreement before data cleaning
keys = list(annotations1.keys())
labels1 = [annotations1[key]["label"] for key in keys]
labels2 = [annotations2[key]["label"] for key in keys]
kappa = cohen_kappa_score(labels1, labels2)
print(kappa)

0.37623762376237624


In [6]:
# Clean data
skipped_keys = set()
for annotations in [annotations1, annotations2]:
    for key in keys:
        # Treat neutral as correct
        if annotations[key]["label"] == "Both / Neutral / Ambiguous":
            annotations[key]["label"] = "Correct Sense"
        # Treat bad translations as wrong
        if annotations[key]["label"] == "Translation too bad to tell / Third sense":
            annotations[key]["label"] = "Wrong Sense"
        # Skip bad samples
        if annotations[key]["label"] == "Bad sample / Ill-defined senses":
            skipped_keys.add(key)
for annotations in [annotations1, annotations2]:
    for key in skipped_keys:
        if key in annotations:
            del annotations[key]
print(f"Number of annotations: {len(annotations1)} + {len(annotations2)}")

Number of annotations: 96 + 96


In [7]:
# Inter-annotator agreement after data cleaning
keys = list(annotations1.keys())
labels1 = [annotations1[key]["label"] for key in keys]
labels2 = [annotations2[key]["label"] for key in keys]
kappa = cohen_kappa_score(labels1, labels2)
print(kappa)

0.9175257731958762


In [8]:
# Merge annotations
annotations = list(itertools.chain(annotations1.values(), annotations2.values()))

In [9]:
# Load task
mucow_ende = MucowContrastiveConditioningTask(
    tgt_language="de",
    evaluator_model=None,
)

In [10]:
# Create list of annotated (= uncovered) samples
uncovered_samples = []
all_samples_dict = {(sample.src_sentence, sample.src_word): sample for sample in mucow_ende.samples}
for annotation in annotations:
    try:
        sample = all_samples_dict[(annotation["Source Sentence"], annotation["Word"])]
    except KeyError:  # Google Sheets removes leading apostrophe
        sample = all_samples_dict.get(("'" + annotation["Source Sentence"], annotation["Word"]), None)
    if sample is None:
        continue
    sample._gold_label = annotation["label"] == "Correct Sense"
    sample.translation = annotation["Translation"]
    uncovered_samples.append(sample)
print("Full number of uncovered samples: ", len(uncovered_samples))

Full number of uncovered samples:  192


In [11]:
# Create list of unannotated (= covered) samples based on log file
num_samples = 0
covered_samples = []
all_samples_dict = {(sample.src_sentence, sample.src_word): sample for sample in mucow_ende.samples}
with jsonlines.open(pattern_matching_log_path) as f:
    for line in f:
        sample = all_samples_dict.get((line["sentence"], line["src_word"]), None)
        if sample is None:
            continue  # Contrastive conditioning not applicable
        if line["corpus"] == "opensubs":  # Only evaluate on in-domain samples because they have higher quality
            continue
        num_samples += 1
        if line["is_unknown"]:  # = uncovered
            continue
        sample._gold_label = line["is_correct"]
        sample.translation = line["translation"]
        covered_samples.append(sample)
random.seed(42)
coverage = len(covered_samples) / num_samples
print("Proportion of covered samples", coverage)
print("Full number of covered samples: ", len(covered_samples))

Proportion of covered samples 0.7886486486486487
Full number of covered samples:  1459


In [12]:
# Sample a proportionate amount of covered samples
_covered_samples = []
for _ in range(int(len(uncovered_samples) * (1 / (1 - coverage)))):
    _covered_samples.append(random.choice(covered_samples))
covered_samples = _covered_samples
print(f"Testing on {len(covered_samples)} covered samples and {len(uncovered_samples)} uncovered samples")

Testing on 908 covered samples and 192 uncovered samples


In [13]:
# Evaluate classic MuCoW
# Count all covered samples as agreements; judge all unknown samples as incorrect translations
num_agreements = len(covered_samples) + sum(1 for sample in uncovered_samples if not sample._gold_label)
proportion_of_agreement = num_agreements / (len(covered_samples) + len(uncovered_samples))
print("Proportion of agreement: ", proportion_of_agreement)

Proportion of agreement:  0.8381818181818181


In [14]:
# Run contrastive conditioning
evaluator_model = load_sota_evaluator("de")

mucow_ende.samples = uncovered_samples + covered_samples
mucow_ende.categories = {sample.category for sample in mucow_ende.samples}
mucow_ende.category_wise_weighting = True
mucow_ende.evaluator_model = evaluator_model

translations = DictTranslationModel({sample.src_sentence: sample.translation for sample in mucow_ende.samples})
contrastive_conditioning_weighted_evaluated_samples = mucow_ende.evaluate(translations).samples

Using cache found in /home/user/vamvas/.cache/torch/hub/pytorch_fairseq_master
Loading codes from /home/user/vamvas/.cache/torch/pytorch_fairseq/0695ef328ddefcb8cbcfabc3196182f59c0e41e0468b10cc0db2ae9c91881fcc.bb1be17de4233e13870bd7d6065bfdb03fca0a51dd0f5d0b7edf5c188eda71f1/bpecodes ...
Read 30000 codes from the codes file.
100%|████████████████████████████████████████████████████████████| 1100/1100 [04:26<00:00,  4.13it/s]


In [15]:
# Create unweighted contrastive conditioning samples
contrastive_conditioning_unweighted_evaluated_samples = deepcopy(contrastive_conditioning_weighted_evaluated_samples)
for sample in contrastive_conditioning_unweighted_evaluated_samples:
    sample.weight = 1

In [16]:
# Evaluate contrastive conditioning
for evaluated_samples in [
    contrastive_conditioning_unweighted_evaluated_samples,
    contrastive_conditioning_weighted_evaluated_samples,
]:
    predicted_labels = []
    gold_labels = []
    weights = []
    for sample in evaluated_samples:
        gold_labels.append(int(sample._gold_label))
        predicted_labels.append(int(sample.is_correct))
        weights.append(getattr(sample, "weight", 1))
    class_labels = [0, 1]
    target_names = ["Wrong", "Correct"]
    print(classification_report(
        y_true=gold_labels,
        y_pred=predicted_labels,
        labels=class_labels,
        target_names=target_names,
        sample_weight=weights,
        zero_division=True,
        digits=3,
    ))

              precision    recall  f1-score   support

       Wrong      0.278     0.288     0.283     146.0
     Correct      0.890     0.886     0.888     954.0

    accuracy                          0.806    1100.0
   macro avg      0.584     0.587     0.585    1100.0
weighted avg      0.809     0.806     0.808    1100.0

              precision    recall  f1-score   support

       Wrong      0.236     0.190     0.211   27443.0
     Correct      0.927     0.944     0.935  298648.0

    accuracy                          0.880  326091.0
   macro avg      0.582     0.567     0.573  326091.0
weighted avg      0.869     0.880     0.874  326091.0

